In [185]:
import numpy as np
import galois
import itertools
GF = galois.GF(2)

In [584]:
# https://scialert.net/fulltext/?doi=tasr.2012.929.934
def girth(H, g):
    # girth can only be even in bipartite graphs
    combs = list(itertools.combinations(np.arange(H.shape[0]), g//2))
    for c in combs:
        w = np.zeros(H.shape[1])
        for r in range(g//2):
            w += np.array(H[c[r]])
        if (np.count_nonzero(w == 2) == g//2):
            return True
    return False

In [653]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 24
deg_c = 4 # w_r. Every check has this many bits in it, d
deg_v = 3 # w_c. Every bit is in this many checks, c
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), np.int64)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            # if (girth(H, 4)):
            #     H[cs[c_ind]][vs[v_ind]] = 0
            #     break
            vs = np.delete(vs, v_ind)
            cs = np.delete(cs, c_ind)

H = GF(H)

In [654]:
# np.savetxt('./good_ldpc_codes/16_4_3.txt', H, fmt="%d")

In [655]:
while (girth(H, 4)):
    arange = np.arange(H.shape[0])
    i = np.random.choice(arange, replace=False)
    j = np.random.choice(arange, replace=False)
    bits_i = np.where(H[i])[0]
    bits_j = np.where(H[j])[0]
    i_ind = np.random.choice(bits_i)
    j_ind = np.random.choice(bits_j)
    
    if (i_ind == j_ind or i_ind in bits_j or j_ind in bits_i): 
        continue
    
    v = GF(np.zeros(H.shape[1], dtype=np.int8))
    v[i_ind] = v[j_ind] = 1
    H[i] += v
    H[j] += v


In [656]:
def rref(H):
    r = 0
    for c in range(k, n):
        if (H[r, c] == 0):
            # need to swap
            for r2 in range(r+1, num_checks):
                if (H[r2, c] == 1):
                    temp = H[r, :].copy()
                    H[r, :] = H[r2, :]
                    H[r2, :] = temp
                    break
                    
        if (H[r, c] == 0):
            print('H is singular')

        # cancel out other rows
        for r2 in range(r+1, num_checks):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        # # back substitution
        for r2 in range(0, r):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        r += 1
    return H

rref_H = GF(rref(GF(H.copy())))

G = GF(np.hstack((GF(np.eye(k, dtype=np.int64)), rref_H[:,0:k].T)))
print(np.any(H @ G.T))

H is singular
H is singular
True
The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [652]:
for i in range(2**k):
    m = [int(j) for j in bin(i)[2:].zfill(k)]
    c = GF(m) @ G
    print(np.array(c), np.count_nonzero(c))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 0
[0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1] 8
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0] 4
[0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1] 10
[0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0] 10
[0 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 1] 12
[0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 0] 10
[0 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1] 10
[0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1] 8
[0 0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0] 8
[0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 1 0 0 1] 10
[0 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0] 12
[0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1] 12
[0 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0] 14
[0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1] 10
[0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0] 14
[0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1] 12
[0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0] 10
[0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1] 1

In [568]:
def flip(y):
    def determine_flippable(w):
        flippable = []
        for v in range(H.shape[1]):
            checks = np.nonzero(H[:, v])[0]
            unsatisfied = np.sum([x(j, w) for j in checks])
            deg = len(checks)
            if (2 * unsatisfied >= deg):
                flippable.append(v)
        return flippable

    # evaluates the lth parity check with recieved vector y
    x = lambda l, y: np.sum([y[k] for k in np.where(H[l])]) % 2 

    w = y.copy()
    flippable = determine_flippable(w)
    while flippable:
        z = GF(np.zeros(H.shape[1], dtype=np.int8))
        z[flippable[0]] = 1
        w += z
        flippable = determine_flippable(w)

    if np.any(H @ w):
        return "FAIL"
    else:
        # where w is the deduced word
        return w

In [569]:
def flip_alt(y):
    def calc_contraints(w):
        # for each constraint, determine if it's satisfied
        constraints = np.array([int(h @ w) for _, h in enumerate(H)])
        # initialize S_0, ..., S_c to empty sets
        constraint_counts = [[] for _ in range(deg_v + 1)]
        # for each variable, count the number of unsatisfied constraints in which it appears
        for i in range(H.shape[1]):
            constraint_counts[np.sum(constraints[np.where(H[:,i])[0]])].append(i)
        return constraint_counts

    def check_termination(counts):
        for i in range(int(np.ceil(deg_v/2)), deg_v + 1):
            if (constraint_counts[i]):
                return False
        return True

    w = y.copy()
    constraint_counts = calc_contraints(w)[::-1]
    while not check_termination(constraint_counts):
        print(w, constraint_counts[::-1])
        for i, _ in enumerate(constraint_counts):
            if (constraint_counts[i]):
                # choose a variable from set S_i
                v = np.random.choice(constraint_counts[i], 1)[0]
                f = GF(np.zeros(w.shape, dtype=np.int8))
                f[v] = 1
                # flip the variable v
                w = w + f
                break
            
        constraint_counts = calc_contraints(w)
    return w

In [570]:
p = 0.01

res = []
for i in range(10000):
    e = GF([1 if np.random.uniform() < p else 0 for i in range(H.shape[1])])
    syndrome = flip(e)
    if (type(syndrome) == str):
        res.append(0)
    else:
        if (np.sum(np.array(syndrome)) == 0):
            res.append(1)
        else:
            res.append(0)